In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
def smooth_ts(original_data, window_len):
    smoothed_df = pd.DataFrame()
    for i, n in enumerate(np.unique(original_data['unit_number'])):
        temp_df = original_data[original_data['unit_number'] == n].copy()
        temp_df.drop(columns=['unit_number', 'time_in_cycles', 'RUL'], inplace=True)
        temp_df = temp_df.rolling(window_len).mean()
        smoothed_df = pd.concat([smoothed_df, temp_df], axis=0)
    smoothed_df.rename(columns={name: name + '_mv_avg' for name in smoothed_df.columns}, inplace=True)
    data = pd.concat([original_data, smoothed_df], axis=1)
    #data.dropna(inplace=True)
    return data


def add_rsi(original_data, cols, window_len=21):
    rsi = pd.DataFrame()
    for i, n in enumerate(np.unique(original_data['unit_number'])):
        init_data = original_data[original_data['unit_number'] == n].copy()
        init_data = init_data[cols]
        init_data.drop(columns=['unit_number', 'time_in_cycles', 'RUL'], inplace=True)        
        delta = init_data.diff()
        up, down = delta.clip(lower=0), delta.clip(upper=0).abs()
        roll_up = up.rolling(window=window_len).sum() / window_len
        roll_down = down.rolling(window=window_len).sum().abs() / window_len
        rs = roll_up / roll_down
        rsi_temp = 100 / (1 + rs)
        if i ==0:
            rsi = rsi_temp
        else:
            rsi = pd.concat((rsi,rsi_temp),axis =0)
        
        
    rsi.rename(columns={name: name + '_rsi' for name in rsi.columns}, inplace=True)
    data = pd.concat([original_data, rsi], axis=1)
    #data.dropna(inplace=True)
    return data

def add_BB(original_data, cols):
    bb = pd.DataFrame()
    for i, n in enumerate(np.unique(original_data['unit_number'])):
        init_data = original_data[original_data['unit_number'] == n].copy()
        #init_data = init_data[cols]
        init_data.drop(columns=['unit_number', 'time_in_cycles', 'RUL'], inplace=True)
        sma = init_data.rolling(window=20).mean()
        ub = sma + 2 * init_data.rolling(window=20).std()
        lb = sma - 2 * init_data.rolling(window=20).std()
        pbb = (init_data - lb) / (ub - lb)
        if i ==0:
            bb = pbb
        else:
            bb = pd.concat((bb,pbb),axis =0)
        
        #bb = pd.concat([bb, pbb], axis=0)
    bb.rename(columns={name: name + '_pbb' for name in bb.columns}, inplace=True)
    data = pd.concat([original_data, bb], axis=1)
    #data.dropna(inplace=True)
    return data


def add_CCI(original_data, ndays=20):
    cci_list = []
    for i, n in enumerate(np.unique(original_data['unit_number'])):
        init_data = original_data[original_data['unit_number'] == n].copy()
        init_data['high'] = init_data.max(axis=1)
        init_data['low'] = init_data.min(axis=1)
        init_data['close'] = init_data.mean(axis=1)
        TP = (init_data['high'] + init_data['low'] + init_data['close']) / 3
        CCI = (TP - TP.rolling(ndays).mean()) / (0.015 * TP.rolling(ndays).std())
        cci_list.append(CCI)

    cci = pd.concat(cci_list, axis=0)
    cci.name = 'CCI'
    cci_df = cci.to_frame()
    cci_df.rename(columns={'CCI': 'CCI_cci'}, inplace=True)
    
    data = pd.concat([original_data, cci_df], axis=1)
    #data.dropna(inplace=True)
    return data

def add_stochastic_oscillator(original_data, window_len=20):
    stoch_k_list = []
    stoch_d_list = []
    
    for n in np.unique(original_data['unit_number']):
        init_data = original_data[original_data['unit_number'] == n].copy()
        init_data['high'] = init_data.max(axis=1)
        init_data['low'] = init_data.min(axis=1)
        init_data['close'] = init_data.mean(axis=1)
        
        L14 = init_data['low'].rolling(window=window_len).min()
        H14 = init_data['high'].rolling(window=window_len).max()
        stoch_k_temp = 100 * ((init_data['close'] - L14) / (H14 - L14))
        stoch_d_temp = stoch_k_temp.rolling(window=3).mean()  # Smoothed %K to get %D
        
        stoch_k_list.append(stoch_k_temp)
        stoch_d_list.append(stoch_d_temp)
    
    stoch_k = pd.concat(stoch_k_list, axis=0)
    stoch_d = pd.concat(stoch_d_list, axis=0)
    
    stoch_k.name = 'stoch_k'
    stoch_d.name = 'stoch_d'
    
    data = pd.concat([original_data, stoch_k, stoch_d], axis=1)
    #data.dropna(inplace=True)
    return data

def add_rolling_std(original_data, cols, window_len = 14):
    bb = pd.DataFrame()
    for i, n in enumerate(np.unique(original_data['unit_number'])):
        init_data = original_data[original_data['unit_number'] == n].copy()
        init_data = init_data[cols]
        init_data.drop(columns=['unit_number', 'time_in_cycles', 'RUL'], inplace=True)
        stdev = init_data.rolling(window=window_len).std()
        if i ==0:
            rstdev = stdev
        else:
            rstdev = pd.concat((rstdev,stdev),axis =0)
    rstdev.rename(columns={name: name + '_std' for name in rstdev.columns}, inplace=True)
    data = pd.concat([original_data, rstdev], axis=1)
    data.dropna(inplace=True)
    return data

def calculate_score(estimated_rul, true_rul, a1=10, a2=13):
    score = []
    for i in range(len(true_rul)):
        d = estimated_rul[i] - true_rul[i]
        if d < 0:
            score.append(np.exp(-d / a1) - 1)
        else:
            score.append(np.exp(d / a2) - 1)
    return np.sum(score)

In [3]:
############### Data Pre-Processing ##########################################################################   

data_set = 1      #specify which dataset to read
train_file_path = 'train_FD00' + str(data_set) + '.txt'
test_file_path = 'test_FD00' + str(data_set) + '.txt'
rul_file_path = 'RUL_FD00' + str(data_set) + '.txt'


headers = ['unit_number','time_in_cycles','setting_1','setting_2','setting_3','T2','T24','T30','T50','P2','P15','P30','Nf',
           'Nc','epr','Ps30','phi','NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

# Read the training & test data
train_data = pd.read_csv(train_file_path, delim_whitespace=True, header=None, names=headers)
test_data = pd.read_csv(test_file_path, delim_whitespace=True, header=None, names=headers)
rul_data = pd.read_csv(rul_file_path, header=None, names=['RUL'])
max_cycle_train = train_data.groupby('unit_number')['time_in_cycles'].max().reset_index()# Calculate the maximum cycle number for each unit in the training data
max_cycle_train.columns = ['unit_number', 'max_cycle']
train_data = train_data.merge(max_cycle_train, on='unit_number', how='left')# Merge the max_cycle_train dataframe with the train data to calculate the RUL for each entry
train_data['RUL'] = train_data['max_cycle'] - train_data['time_in_cycles']# Calculate the RUL for each entry in the train data
train_data = train_data.drop(columns=['max_cycle'])# Drop the temporary max_cycle column
rul_data.columns = ['RUL']
rul_data['unit_number'] = rul_data.index + 1
max_cycle_test = test_data.groupby('unit_number')['time_in_cycles'].max().reset_index()# Calculate the maximum cycle number for each unit in the test data
max_cycle_test.columns = ['unit_number', 'max_cycle']
test_data = test_data.merge(max_cycle_test, on='unit_number', how='left')# Merge the max_cycle_test dataframe with the test data to calculate the RUL for each entry

# Assign the RUL values to the test data based on unit number
# The provided RUL is at the end of the data collection, add this RUL to the difference between max cycle and current cycle
test_data['RUL'] = test_data.apply(lambda row: rul_data.loc[rul_data['unit_number'] == row['unit_number'], 'RUL'].values[0] + (row['max_cycle'] - row['time_in_cycles']), axis=1)
test_data = test_data.drop(columns=['max_cycle'])# Drop the temporary max_cycle column


######### Calculate VIF to determine which variables to keep  
vif_thresh = 10
features = train_data.drop(columns=['unit_number', 'time_in_cycles', 'RUL'])# Drop unnecessary columns
vif_data = pd.DataFrame()
scaler = StandardScaler()  # Standardize the data
x = scaler.fit_transform(features)
standardized_data = pd.DataFrame(x, columns=features.columns) # Create a DataFrame for the standardized data
var_df = features.var()
#non_zero_variance_features = standardized_data.loc[:, standardized_data.var() != 0] # Remove features with zero variance
non_zero_variance_features = standardized_data.loc[:, features.var() > 0.001]
# Calculate VIF for each feature
vif_data = pd.DataFrame() 
vif_data['Feature'] = non_zero_variance_features.columns
vif_data['VIF'] = [variance_inflation_factor(non_zero_variance_features.values, i) for i in range(non_zero_variance_features.shape[1])]
vif_data['Feature'] = non_zero_variance_features.columns
vif_data['VIF'] = [variance_inflation_factor(non_zero_variance_features.values, i) for i in range(non_zero_variance_features.shape[1])]

if data_set in [2,4]:
    keep = ['unit_number', 'time_in_cycles','RUL']+list(vif_data['Feature'][vif_data['VIF']<= vif_thresh])
    include= ['Nf','Nc','NRf','NRc','T24','T30','P30','W32','phi','BPR','htBleed']
    for i in include:
        if not i in keep:
            keep.append(i)
else:
    keep = ['unit_number','time_in_cycles','setting_1','T24','T30','T50','P30','Nf','Nc','Ps30','phi','NRf','NRc','BPR','htBleed','W31','W32','RUL']

############### Create training and test sets with added time series/ indicator variables ######################
train_data = train_data[keep]
test_data = test_data[keep]

# Add RSI values
train_data = add_rsi(train_data,cols=keep, window_len=21)
test_data = add_rsi(test_data,cols=keep, window_len=21)

# Add BB values
train_data = add_BB(train_data,keep)
test_data = add_BB(test_data,keep)

# #Add CCI values
train_data = add_CCI(train_data, ndays=20)
test_data = add_CCI(test_data, ndays=20)

# Add Stochastic Oscillator values
train_data = add_stochastic_oscillator(train_data, window_len=14)
test_data = add_stochastic_oscillator(test_data,  window_len=14)


train_data.dropna(inplace=True)
test_data.dropna(inplace=True)


# Prepare the data for training
X_train = train_data.drop(columns=['unit_number', 'time_in_cycles', 'RUL'])
y_train = train_data[['unit_number', 'time_in_cycles', 'RUL']]
X_test = test_data.drop(columns=['unit_number', 'time_in_cycles', 'RUL'])
y_test = test_data[['unit_number', 'time_in_cycles', 'RUL']]

FileNotFoundError: [Errno 2] No such file or directory: 'train_FD001.txt'

In [ ]:
# Load and preprocess the data (assuming you have the data loaded in X_train, y_train, and test_data)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Filter test data to get the rows with the maximum time_in_cycles for each unit_number
max_cycles = test_data.groupby('unit_number')['time_in_cycles'].max().reset_index()
test_max_cycles = pd.merge(test_data, max_cycles, on=['unit_number', 'time_in_cycles'], how='inner')

# Scale the test data
X_test_max_scaled = scaler.transform(test_max_cycles.drop(columns=['unit_number', 'time_in_cycles', 'RUL']))

model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))  # Output layer for regression task
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train['RUL'], epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(X_test_max_scaled, test_max_cycles['RUL'])
print(f'Test Mean Absolute Error: {mae:.4f}')

# Make predictions
predictions = model.predict(X_test_max_scaled)
predictions[predictions <0] = 0

# Calculate the R² value
r2 = r2_score(test_max_cycles['RUL'], predictions)
print(f'R² Value: {r2:.4f}')

# Calculate the score using the custom score function
model_score = calculate_score(predictions.flatten(), test_max_cycles['RUL'].values)

print(f'Model Score: {model_score:.4f}')
plt.figure(figsize=(10, 6))
plt.plot(test_max_cycles['unit_number'], test_max_cycles['RUL'], label='True RUL', marker='o')
plt.plot(test_max_cycles['unit_number'], predictions, label='Predicted RUL', marker='x')
plt.title('FD003 True vs Predicted RUL')
plt.xlabel('Unit Number')
plt.ylabel('RUL')
plt.legend()
plt.show()


In [7]:
import joblib
import h5py

In [8]:
# Save the model using the recommended Keras format
model.save('trained_model.keras')

# Save the scaler
joblib.dump(scaler, 'scaler.save')

['scaler.save']

In [10]:
# Load the model
model = load_model('trained_model.keras')
 
# Load the scaler
scaler = joblib.load('scaler.save')
 
# Assuming X_test is your test data
X_test_scaled = scaler.transform(X_test)
predictions = model.predict(X_test_scaled)

190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 629us/step


In [11]:
import shutil

# Create a zip file
shutil.make_archive('model_and_scaler', 'zip', '.', 'trained_model.keras')
shutil.make_archive('model_and_scaler', 'zip', '.', 'scaler.save')

'C:\\Users\\baris\\Practicum\\model_and_scaler.zip'